### Extração de dados da PIM-PF/IBGE

Esse código importa as séries atualizadas da PIM-PF diretamente da API do SIDRA e organiza os dados para facilitar os gráficos e as tabelas no formato do Informativo do Necat/UFSC.

**Se você estiver utilizando o Google Colab, será necessário instalar o pacote sidrapy.** Execute a primeira célula para instalar esse pacote.

In [ ]:
!pip install sidrapy

In [28]:
import pandas as pd
import sidrapy as sp
import numpy as np
from google.colab import files

# Dados do Brasil:

Importando os dados da produção física industrial (PFI) do Brasil com ajuste sazonal (Gráfico 1):

In [ ]:
PFIBR_raw = sp.get_table(table_code='8888', territorial_level='1', ibge_territorial_code='all', variable='12607',period='last%2055', classification='544/129314')

#Organizando as linhas e excluindo a última observação
PFIBR_raw.columns = PFIBR_raw.iloc[0]
PFIBR_raw = PFIBR_raw.iloc[1:,:]

#Selecionando as colunas de interesse e renomeando
PFIBR = PFIBR_raw.rename(columns = {"Valor":"PFIBR", "Mês (Código)":"date"})[['PFIBR', 'date']]

#Convertendo o valor do índice para float e tratando o formato data
PFIBR['PFIBR'] = PFIBR['PFIBR'].astype(float)

PFIBR.index = pd.to_datetime(PFIBR['date'],format = "%Y%m")
PFIBR = PFIBR.drop(columns = ['date'])

#Indicando jan/2020=100 e excluindo a coluna com os valores jan/2012=100

PFIBR['PFIBR_Fixa'] = PFIBR/PFIBR['PFIBR'].iloc[0]*100
PFIBR = PFIBR.drop(columns = ['PFIBR'])

PFIBR.head()

Importando as variações da PFI de acordo com as bases de comparação do IBGE (Tabela 1):

In [ ]:
#Importando e organizando a série mensal com ajuste sazonal
var1BR_raw = sp.get_table(table_code='8888', territorial_level='1', ibge_territorial_code='all', variable='11601', period='last%2012', classification='544/129314')

var1BR_raw.columns = var1BR_raw.iloc[0]
var1BR_raw = var1BR_raw.iloc[1:,:]

var1BR = var1BR_raw.rename(columns = {"Valor":"Mensal com ajuste sazonal", 'Mês (Código)': 'date'})[['Mensal com ajuste sazonal', 'date']]

var1BR['Mensal com ajuste sazonal'] = var1BR['Mensal com ajuste sazonal'].astype(float)

var1BR.index = pd.to_datetime(var1BR['date'], format = "%Y%m")
var1BR = var1BR.drop(columns = ['date'])

#Importando e organizando a série mensal em relação ao mês anterior
var2BR_raw = sp.get_table(table_code='8888', territorial_level='1', ibge_territorial_code='all', variable='11602', period='last%2012', classification='544/129314')

var2BR_raw.columns = var2BR_raw.iloc[0]
var2BR_raw = var2BR_raw.iloc[1:,:]

var2BR = var2BR_raw.rename(columns = {"Valor":"Mês/Mês ano anterior", 'Mês (Código)': 'date'})[['Mês/Mês ano anterior', 'date']]

var2BR['Mês/Mês ano anterior'] = var2BR['Mês/Mês ano anterior'].astype(float)

var2BR.index = pd.to_datetime(var2BR['date'], format = "%Y%m")
var2BR = var2BR.drop(columns = ['date'])

#Importando e organizando a série do acumulado anual
var3BR_raw = sp.get_table(table_code='8888', territorial_level='1', ibge_territorial_code='all', variable='11603', period='last%2012', classification='544/129314')

var3BR_raw.columns = var3BR_raw.iloc[0]
var3BR_raw = var3BR_raw.iloc[1:,:]

var3BR = var3BR_raw.rename(columns = {"Valor":"Acumulado anual", 'Mês (Código)': 'date'})[['Acumulado anual', 'date']]

var3BR['Acumulado anual'] = var3BR['Acumulado anual'].astype(float)

var3BR.index = pd.to_datetime(var3BR['date'], format = "%Y%m")
var3BR = var3BR.drop(columns = ['date'])

#Importando e organizando a série do acumulado de doze meses
var4BR_raw = sp.get_table(table_code='8888', territorial_level='1', ibge_territorial_code='all', variable='11604', period='last%2012', classification='544/129314')

var4BR_raw.columns = var4BR_raw.iloc[0]
var4BR_raw = var4BR_raw.iloc[1:,:]

var4BR = var4BR_raw.rename(columns = {"Valor":"Acumulado doze meses", 'Mês (Código)': 'date'})[['Acumulado doze meses', 'date']]

var4BR['Acumulado doze meses'] = var4BR['Acumulado doze meses'].astype(float)

var4BR.index = pd.to_datetime(var4BR['date'], format = "%Y%m")
var4BR = var4BR.drop(columns = ['date'])

#Organizando as quatro séries no mesmo dataframe
PFIBR_vars = pd.merge(var1BR, var2BR, left_index=True, right_index=True)
PFIBR_vars = pd.merge(PFIBR_vars, var3BR, left_index=True, right_index=True)
PFIBR_vars = pd.merge(PFIBR_vars, var4BR, left_index=True, right_index=True)

PFIBR_vars

Importando as séries da PFI desagregada por categorias de uso (Gráfico 2):

In [ ]:
#Importando o índice do segmento de bens de capital
uso1_raw = sp.get_table(table_code='8887', territorial_level='1', ibge_territorial_code='all', variable = '12607', period='last%2055', classification='543/129278')

uso1_raw.columns = uso1_raw.iloc[0]
uso1_raw = uso1_raw.iloc[1:,:]

uso1 = uso1_raw.rename(columns = {"Valor":"Capital", "Mês (Código)":"date"})[['Capital', 'date']]

uso1['Capital'] = uso1['Capital'].astype(float)

uso1.index = pd.to_datetime(uso1['date'],format = "%Y%m")
uso1 = uso1.drop(columns = ['date'])

uso1['Capital_Fixa'] = uso1/uso1['Capital'].iloc[0]*100
uso1 = uso1.drop(columns = ['Capital'])

#Importando o índice do segmento de bens intermediários
uso2_raw = sp.get_table(table_code='8887', territorial_level='1', ibge_territorial_code='all', variable = '12607', period='last%2055', classification='543/129283')

uso2_raw.columns = uso2_raw.iloc[0]
uso2_raw = uso2_raw.iloc[1:,:]

uso2 = uso2_raw.rename(columns = {"Valor":"Intermediários", "Mês (Código)":"date"})[['Intermediários', 'date']]

uso2['Intermediários'] = uso2['Intermediários'].astype(float)

uso2.index = pd.to_datetime(uso2['date'],format = "%Y%m")
uso2 = uso2.drop(columns = ['date'])

uso2['Intermediários_Fixa'] = uso2/uso2['Intermediários'].iloc[0]*100
uso2 = uso2.drop(columns = ['Intermediários'])

#Importando o índice do segmento de bens de consumo duráveis
uso3_raw = sp.get_table(table_code='8887', territorial_level='1', ibge_territorial_code='all', variable = '12607', period='last%2055', classification='543/129301')

uso3_raw.columns = uso3_raw.iloc[0]
uso3_raw = uso3_raw.iloc[1:,:]

uso3 = uso3_raw.rename(columns = {"Valor":"Duráveis", "Mês (Código)":"date"})[['Duráveis', 'date']]

uso3['Duráveis'] = uso3['Duráveis'].astype(float)

uso3.index = pd.to_datetime(uso3['date'],format = "%Y%m")
uso3 = uso3.drop(columns = ['date'])

uso3['Duráveis_Fixa'] = uso3/uso3['Duráveis'].iloc[0]*100
uso3 = uso3.drop(columns = ['Duráveis'])

#Importando o índice do segmento de bens de consumo semi e não duráveis
uso4_raw = sp.get_table(table_code='8887', territorial_level='1', ibge_territorial_code='all', variable = '12607', period='last%2055', classification='543/129305')

uso4_raw.columns = uso4_raw.iloc[0]
uso4_raw = uso4_raw.iloc[1:,:]

uso4 = uso4_raw.rename(columns = {"Valor":"Semi e não duráveis", "Mês (Código)":"date"})[['Semi e não duráveis', 'date']]

uso4['Semi e não duráveis'] = uso4['Semi e não duráveis'].astype(float)

uso4.index = pd.to_datetime(uso4['date'],format = "%Y%m")
uso4 = uso4.drop(columns = ['date'])

uso4['Semi e não duráveis_Fixa'] = uso4/uso4['Semi e não duráveis'].iloc[0]*100
uso4 = uso4.drop(columns = ['Semi e não duráveis'])

#Organizando as quatro séries no mesmo dataframe
uso = pd.merge(uso1, uso2, left_index=True, right_index=True)
uso = pd.merge(uso, uso3, left_index=True, right_index=True)
uso = pd.merge(uso, uso4, left_index=True, right_index=True)

uso.head()

Importando as variações da PFI desagregadas por categorias de uso (Tabela 2):

In [ ]:
#Importando a série mensal com ajuste sazonal e agrupando
usovar1_raw = sp.get_table(table_code = '8887', territorial_level='1', ibge_territorial_code='all', variable = '11601', period='last%201', classification='543/all')

usovar1_raw.columns = usovar1_raw.iloc[0]
usovar1_raw = usovar1_raw.iloc[1:,:]

usovar1_raw = usovar1_raw[usovar1_raw['Valor'] != '-']

usovar1 = usovar1_raw.rename(columns = {"Valor":"Mensal com ajuste sazonal", "Grandes categorias econômicas": "Categorias de uso"})[['Mensal com ajuste sazonal', 'Categorias de uso']]

usovar1['Mensal com ajuste sazonal'] = usovar1['Mensal com ajuste sazonal'].astype(float)

usovar1.index = usovar1['Categorias de uso']
usovar1 = usovar1.drop(columns = ['Categorias de uso'])

#Importando e organizando a série mensal em relação ao mês anterior
usovar2_raw = sp.get_table(table_code = '8887', territorial_level='1', ibge_territorial_code='all', variable = '11602', period='last%201', classification='543/all')

usovar2_raw.columns = usovar2_raw.iloc[0]
usovar2_raw = usovar2_raw.iloc[1:,:]

usovar2_raw = usovar2_raw[usovar2_raw['Valor'] != '-']

usovar2 = usovar2_raw.rename(columns = {"Valor":"Mês/Mês ano anterior", "Grandes categorias econômicas": "Categorias de uso"})[['Mês/Mês ano anterior', 'Categorias de uso']]

usovar2['Mês/Mês ano anterior'] = usovar2['Mês/Mês ano anterior'].astype(float)

usovar2.index = usovar2['Categorias de uso']
usovar2 = usovar2.drop(columns = ['Categorias de uso'])
usovar2 = usovar2.loc[['1 Bens de capital', '2 Bens intermediários', '3 Bens de consumo', '31 Bens de consumo duráveis', '32 Bens de consumo semiduráveis e não duráveis']]

#Importando e organizando a série do acumulado anual
usovar3_raw = sp.get_table(table_code = '8887', territorial_level='1', ibge_territorial_code='all', variable = '11603', period='last%201', classification='543/all')

usovar3_raw.columns = usovar3_raw.iloc[0]
usovar3_raw = usovar3_raw.iloc[1:,:]

usovar3_raw = usovar3_raw[usovar3_raw['Valor'] != '-']

usovar3 = usovar3_raw.rename(columns = {"Valor":"Acumulado anual", "Grandes categorias econômicas": "Categorias de uso"})[['Acumulado anual', 'Categorias de uso']]

usovar3['Acumulado anual'] = usovar3['Acumulado anual'].astype(float)

usovar3.index = usovar3['Categorias de uso']
usovar3 = usovar3.drop(columns = ['Categorias de uso'])
usovar3 = usovar3.loc[['1 Bens de capital', '2 Bens intermediários', '3 Bens de consumo', '31 Bens de consumo duráveis', '32 Bens de consumo semiduráveis e não duráveis']]

#Importando e organizando a série do acumulado de doze meses
usovar4_raw = sp.get_table(table_code = '8887', territorial_level='1', ibge_territorial_code='all', variable = '11604', period='last%201', classification='543/all')

usovar4_raw.columns = usovar4_raw.iloc[0]
usovar4_raw = usovar4_raw.iloc[1:,:]

usovar4_raw = usovar4_raw[usovar4_raw['Valor'] != '-']

usovar4 = usovar4_raw.rename(columns = {"Valor":"Acumulado doze meses", "Grandes categorias econômicas": "Categorias de uso"})[['Acumulado doze meses', 'Categorias de uso']]

usovar4['Acumulado doze meses'] = usovar4['Acumulado doze meses'].astype(float)

usovar4.index = usovar4['Categorias de uso']
usovar4 = usovar4.drop(columns = ['Categorias de uso'])
usovar4 = usovar4.loc[['1 Bens de capital', '2 Bens intermediários', '3 Bens de consumo', '31 Bens de consumo duráveis', '32 Bens de consumo semiduráveis e não duráveis']]

#Empilhando os dados das variações por categorias de uso
uso_vars = pd.merge(usovar1, usovar2, left_index=True, right_index=True)
uso_vars = pd.merge(uso_vars, usovar3, left_index=True, right_index=True)
uso_vars = pd.merge(uso_vars, usovar4, left_index=True, right_index=True)

uso_vars

Importando as variações mensais da PFI por seções da CNAE (Tabela 3):

In [ ]:
#Importando a série por seção da CNAE e organizando
sec_vars1_raw = sp.get_table(table_code='8888', territorial_level='1', ibge_territorial_code='all', variable='11601', period='last%207', classification='544/56689,129317,129318,129319,129320,129321,129322,129323,129324,129325,129326,129330,129331,129332,129333,129334,129335,129336,129337,129338,129339,129340,129341,129342')

sec_vars1_raw.columns = sec_vars1_raw.iloc[0]
sec_vars1_raw = sec_vars1_raw.iloc[1:,:]

sec_vars1 = sec_vars1_raw.rename(columns = {'Valor': 'Mensal com ajuste sazonal', 'Seções e atividades industriais (CNAE 2.0)': 'CNAE', 'Mês (Código)': 'Mês_c'})[['Mensal com ajuste sazonal', 'CNAE', 'Mês_c']]
sec_vars1['Mensal com ajuste sazonal'] = sec_vars1['Mensal com ajuste sazonal'].astype(float)

sec_vars1.index = sec_vars1['CNAE']
sec_vars1 = sec_vars1.drop(columns = ['CNAE'])
sec_vars1 = sec_vars1.sort_index()

#Pivotando o dataframe para o formato wide
sec_vars1 = sec_vars1.pivot(columns='Mês_c', values='Mensal com ajuste sazonal')

sec_vars1.head()

Importando as variações acumuladas da PFI por seções da CNAE (Gráfico 3):

In [ ]:
#Importando a série por seção da CNAE e organizando
sec_vars2_raw = sp.get_table(table_code='8888', territorial_level='1', ibge_territorial_code='all', variable='11603', period='last%201', classification='544/56689,129317,129318,129319,129320,129321,129322,129323,129324,129325,129326,129330,129331,129332,129333,129334,129335,129336,129337,129338,129339,129340,129341,129342')

sec_vars2_raw.columns = sec_vars2_raw.iloc[0]
sec_vars2_raw = sec_vars2_raw.iloc[1:,:]

sec_vars2 = sec_vars2_raw.rename(columns = {'Valor': 'Acumulado frente ano anterior', 'Seções e atividades industriais (CNAE 2.0)': 'CNAE'})[['Acumulado frente ano anterior', 'CNAE']]
sec_vars2['Acumulado frente ano anterior'] = sec_vars2['Acumulado frente ano anterior'].astype(float)

#Configurando o índice e ordenando da menor para a maior variação
sec_vars2.index = sec_vars2['CNAE']
sec_vars2 = sec_vars2.drop(columns = ['CNAE'])
sec_vars2 = sec_vars2.sort_values(by='Acumulado frente ano anterior')

sec_vars2.head()

# Dados de Santa Catarina:

Importando os dados da PFI de SC com ajuste sazonal (Gráfico 4):

In [ ]:
PFISC_raw = sp.get_table(table_code='8888', territorial_level='3', ibge_territorial_code='42', variable='12607', period='last%2055', classification='544/129314')

PFISC_raw.columns = PFISC_raw.iloc[0]
PFISC_raw = PFISC_raw.iloc[1:,:]

PFISC = PFISC_raw.rename(columns = {'Valor': "PFISC", 'Mês (Código)': 'date'})[['PFISC', 'date']]

PFISC['PFISC'] = PFISC['PFISC'].astype(float)

PFISC.index = pd.to_datetime(PFISC['date'], format = "%Y%m")
PFISC = PFISC.drop(columns = ['date'])

PFISC['PFISC_Fixa'] = PFISC/PFISC['PFISC'].iloc[0]*100
PFISC = PFISC.drop(columns = ['PFISC'])

PFISC.head()

Importando as variações da PFI de SC de acordo com as bases de comparação do IBGE (Tabela 4):

In [ ]:
#Importando e organizando a série mensal com ajuste sazonal
var1SC_raw = sp.get_table(table_code='8888', territorial_level='3', ibge_territorial_code='42', variable='11601', period='last%2055', classification='544/129314')

var1SC_raw.columns = var1SC_raw.iloc[0]
var1SC_raw = var1SC_raw.iloc[1:,:]

var1SC = var1SC_raw.rename(columns = {"Valor":"Mensal com ajuste sazonal", 'Mês (Código)': 'date'})[['Mensal com ajuste sazonal', 'date']]

var1SC['Mensal com ajuste sazonal'] = var1SC['Mensal com ajuste sazonal'].astype(float)

var1SC.index = pd.to_datetime(var1SC['date'], format = "%Y%m")
var1SC = var1SC.drop(columns = ['date'])

#Importando e organizando a série mensal em relação ao mês anterior
var2SC_raw = sp.get_table(table_code='8888', territorial_level='3', ibge_territorial_code='42', variable='11602', period='last%2055', classification='544/129314')

var2SC_raw.columns = var2SC_raw.iloc[0]
var2SC_raw = var2SC_raw.iloc[1:,:]

var2SC = var2SC_raw.rename(columns = {"Valor":"Mês/Mês ano anterior", 'Mês (Código)': 'date'})[['Mês/Mês ano anterior', 'date']]

var2SC['Mês/Mês ano anterior'] = var2SC['Mês/Mês ano anterior'].astype(float)

var2SC.index = pd.to_datetime(var2SC['date'], format = "%Y%m")
var2SC = var2SC.drop(columns = ['date'])

#Importando e organizando a série do acumulado anual
var3SC_raw = sp.get_table(table_code='8888', territorial_level='3', ibge_territorial_code='42', variable='11603', period='last%2055', classification='544/129314')

var3SC_raw.columns = var3SC_raw.iloc[0]
var3SC_raw = var3SC_raw.iloc[1:,:]

var3SC = var3SC_raw.rename(columns = {"Valor":"Acumulado anual", 'Mês (Código)': 'date'})[['Acumulado anual', 'date']]

var3SC['Acumulado anual'] = var3SC['Acumulado anual'].astype(float)

var3SC.index = pd.to_datetime(var3SC['date'], format = "%Y%m")
var3SC = var3SC.drop(columns = ['date'])

#Importando e organizando a série do acumulado de doze meses
var4SC_raw = sp.get_table(table_code='8888', territorial_level='3', ibge_territorial_code='42', variable='11604', period='last%2012', classification='544/129314')

var4SC_raw.columns = var4SC_raw.iloc[0]
var4SC_raw = var4SC_raw.iloc[1:,:]

var4SC = var4SC_raw.rename(columns = {"Valor":"Acumulado doze meses", 'Mês (Código)': 'date'})[['Acumulado doze meses', 'date']]

var4SC['Acumulado doze meses'] = var4SC['Acumulado doze meses'].astype(float)

var4SC.index = pd.to_datetime(var4SC['date'], format = "%Y%m")
var4SC = var4SC.drop(columns = ['date'])

#Organizando as quatro séries no mesmo dataframe
PFISC_vars = pd.merge(var1SC, var2SC, left_index=True, right_index=True)
PFISC_vars = pd.merge(PFISC_vars, var3SC, left_index=True, right_index=True)
PFISC_vars = pd.merge(PFISC_vars, var4SC, left_index=True, right_index=True)

PFISC_vars

Importando as variações mensais da PFI de SC por seções da CNAE (Tabela 5):

In [ ]:
#Importando a série por seção da CNAE e organizando
sec_varsSC1_raw = sp.get_table(table_code='8888', territorial_level='3', ibge_territorial_code='42', variable='11602', period='last%207', classification='544/56689,129317,129318,129319,129320,129321,129322,129323,129324,129325,129326,129330,129331,129332,129333,129334,129335,129336,129337,129338,129339,129340,129341,129342')

sec_varsSC1_raw.columns = sec_varsSC1_raw.iloc[0]
sec_varsSC1_raw = sec_varsSC1_raw.iloc[1:,:]

sec_varsSC1 = sec_varsSC1_raw.rename(columns = {'Valor': 'Mensal com ajuste sazonal', 'Seções e atividades industriais (CNAE 2.0)': 'CNAE', 'Mês (Código)': 'Mês_c'})[['Mensal com ajuste sazonal', 'CNAE', 'Mês_c']]
sec_varsSC1 = sec_varsSC1[sec_varsSC1['Mensal com ajuste sazonal'] != '-']
sec_varsSC1['Mensal com ajuste sazonal'] = sec_varsSC1['Mensal com ajuste sazonal'].astype(float)

sec_varsSC1.index = sec_varsSC1['CNAE']
sec_varsSC1 = sec_varsSC1.drop(columns = ['CNAE'])
sec_varsSC1 = sec_varsSC1.sort_index()

#Pivotando o dataframe para o formato wide
sec_varsSC1 = sec_varsSC1.pivot(columns='Mês_c', values='Mensal com ajuste sazonal')

sec_varsSC1.head()

Importando as variações acumuladas da PFI de SC por seções da CNAE (Gráfico 5):

In [ ]:
#Importando a série por seção da CNAE e organizando
sec_varsSC2_raw = sp.get_table(table_code='8888', territorial_level='3', ibge_territorial_code='42', variable='11603', period='last%201', classification='544/56689,129317,129318,129319,129320,129321,129322,129323,129324,129325,129326,129330,129331,129332,129333,129334,129335,129336,129337,129338,129339,129340,129341,129342')

sec_varsSC2_raw.columns = sec_varsSC2_raw.iloc[0]
sec_varsSC2_raw = sec_varsSC2_raw.iloc[1:,:]

sec_varsSC2 = sec_varsSC2_raw.rename(columns = {'Valor': 'Acumulado frente ano anterior', 'Seções e atividades industriais (CNAE 2.0)': 'CNAE'})[['Acumulado frente ano anterior', 'CNAE']]
sec_varsSC2 = sec_varsSC2[sec_varsSC2['Acumulado frente ano anterior'] != '-']
sec_varsSC2['Acumulado frente ano anterior'] = sec_varsSC2['Acumulado frente ano anterior'].astype(float)

#Configurando o índice e ordenando da menor para a maior variação
sec_varsSC2.index = sec_varsSC2['CNAE']
sec_varsSC2 = sec_varsSC2.drop(columns = ['CNAE'])
sec_varsSC2 = sec_varsSC2.sort_values(by='Acumulado frente ano anterior')

sec_varsSC2.head()

Baixando os arquivos para os gráficos e as tabelas

In [39]:
#Gráfico 1
PFIBR.to_excel('Graph1.xlsx')
files.download('Graph1.xlsx')

#Tabela 1
PFIBR_vars.to_excel('Tab1.xlsx')
files.download('Tab1.xlsx')

#Gráfico 2
uso.to_excel('Graph2.xlsx')
files.download('Graph2.xlsx')

#Tabela 2
uso_vars.to_excel('Tab2.xlsx')
files.download('Tab2.xlsx')

#Tabela 3
sec_vars1.to_excel('Tab3.xlsx')
files.download('Tab3.xlsx')

#Gráfico 3
sec_vars2.to_excel('Graph3.xlsx')
files.download('Graph3.xlsx')

#Gráfico 4
PFISC.to_excel('Graph4.xlsx')
files.download('Graph4.xlsx')

#Tabela 4
PFISC_vars.to_excel('Tab4.xlsx')
files.download('Tab4.xlsx')

#Tabela 5
sec_varsSC1.to_excel('Tab5.xlsx')
files.download('Tab5.xlsx')

#Gráfico 5
sec_varsSC2.to_excel('Graph5.xlsx')
files.download('Graph5.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>